In [70]:
import sys
from pandas import Series, DataFrame, MultiIndex
import pandas as pd
import numpy as np
from numpy import nan as NA
import csv
import json
from lxml.html import parse
from urllib import request
from lxml import objectify
from io import StringIO

## 读写文本格式的数据


In [2]:
# 含标题
df = pd.read_csv("../resource/ex1.csv")
print(df)

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [4]:
print(pd.read_table("../resource/ex1.csv", sep=","))

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [5]:
# 不含标题
print(pd.read_csv("../resource/ex2.csv", header=None))
print(pd.read_csv("../resource/ex2.csv", 
                  names=['a','b','c','d','message']))


   0   1   2   3      4
0  1   2   3   4  hello
1  5   6   7   8  world
2  9  10  11  12    foo
   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [6]:
# 指定message列作为dataframe的索引
names = ['a','b','c','d','message']
print(pd.read_csv("../resource/ex2.csv",
                  names=names,
                  index_col='message'))

         a   b   c   d
message               
hello    1   2   3   4
world    5   6   7   8
foo      9  10  11  12


In [4]:
# 将多列做成层次化索引
parsed = pd.read_csv("../resource/csv_mindex.csv",
                     index_col=['key1','key2'])
print(parsed)

           value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16


In [6]:
# 不是固定的分隔符分隔字段，使用正则
res = pd.read_table("../resource/ex3.txt",
                    sep='\s+')
print(res)

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [8]:
# 跳过文件的第 1 3 4 行
res = pd.read_csv("../resource/ex4.csv",
                  skiprows=[0,2,3])
print(res)

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [9]:
# 缺失值：NA -1.#IND NULL
res = pd.read_csv("../resource/ex5.csv")
print(res)
print(pd.isnull(res))

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo
   something      a      b      c      d  message
0      False  False  False  False  False     True
1      False  False  False   True  False    False
2      False  False  False  False  False    False


In [15]:
# 此时，值为foo的单元格也为NaN
res = pd.read_csv("../resource/ex5.csv",
                  na_values=['foo'])
print(res)


  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


In [16]:
# 为各列分别指定缺失值
res = pd.read_csv("../resource/ex5.csv",
                  na_values={
                      'message':['foo','NA'],
                      'something':'one'
                  })
print(res)

  something  a   b     c   d message
0       NaN  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


## 逐块读取文本文件

数据量很大时

In [17]:
res = pd.read_csv("../resource/ex6.csv")
print(res)

           one       two     three      four key
0     0.467976 -0.038649 -0.295344 -1.824726   L
1    -0.358893  1.404453  0.704965 -0.200638   B
2    -0.501840  0.659254 -0.421691 -0.057688   G
3     0.204886  1.074134  1.388361 -0.982404   R
4     0.354628 -0.133116  0.283763 -0.837063   Q
...        ...       ...       ...       ...  ..
9995  2.311896 -0.417070 -1.409599 -0.515821   L
9996 -0.479893 -0.650419  0.745152 -0.646038   E
9997  0.523331  0.787112  0.486066  1.093156   K
9998 -0.362559  0.598894 -1.843201  0.887292   G
9999 -0.096376 -1.012999 -0.657431 -0.573315   0

[10000 rows x 5 columns]


In [18]:
# 仅读5行
print(pd.read_csv("../resource/ex6.csv", nrows=5))


        one       two     three      four key
0  0.467976 -0.038649 -0.295344 -1.824726   L
1 -0.358893  1.404453  0.704965 -0.200638   B
2 -0.501840  0.659254 -0.421691 -0.057688   G
3  0.204886  1.074134  1.388361 -0.982404   R
4  0.354628 -0.133116  0.283763 -0.837063   Q


In [21]:
# 逐块读取
chunker = pd.read_csv("../resource/ex6.csv", chunksize=1000)
print(chunker)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value=0)
tot = tot.sort_values(ascending=False)
print(tot[:10])

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64


C:\Users\zgg\AppData\Local\Temp\ipykernel_3484\437882053.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = Series([])


## 将数据写出到文本格式

In [22]:
data = pd.read_csv("../resource/ex5.csv")
print(data)

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo


In [23]:
data.to_csv("../resource/ex5_2.csv")

In [26]:
# 指定分隔符
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [27]:
# 标记空字符串
data.to_csv(sys.stdout, sep='|',na_rep='NULL')

|something|a|b|c|d|message
0|one|1|2|3.0|4|NULL
1|two|5|6|NULL|8|world
2|three|9|10|11.0|12|foo


In [29]:
# 不写行列标签
data.to_csv(sys.stdout, 
            sep='|',
            na_rep='NULL',
            index=False,
            columns=['a','b','c'])

a|b|c
1|2|3.0
5|6|NULL
9|10|11.0


In [31]:
dates = pd.date_range('1/1/2000',periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv("../resource/tseries.csv")

## 手工处理分隔符格式

In [33]:

f = open("../resource/ex7.csv")
reader = csv.reader(f)
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [38]:
lines = list(csv.reader(open("../resource/ex7.csv")))
print(lines)

header, values = lines[0], lines[1:]
print(header, values)

data_dict = {h: v for h, v in zip(header, zip(*values))}
print(data_dict)

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3', '4']]
['a', 'b', 'c'] [['1', '2', '3'], ['1', '2', '3', '4']]
{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}


In [42]:
# 自定义分隔符
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
f = open("../resource/ex7.csv")
reader = csv.reader(f, dialect=my_dialect)
# 也可以直接指定
reader2 = csv.reader(f, delimiter = ';')

In [43]:
# 手动输出分隔符文件
with open("../resource/mydata.csv", 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one','two','three'))
    writer.writerow(('1','2','3'))

## JSON数据

In [45]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [46]:
# json->python
res = json.loads(obj)
print(res)

{'name': 'Wes', 'places_lived': ['United States', 'Spain', 'Germany'], 'pet': None, 'siblings': [{'name': 'Scott', 'age': 25, 'pet': 'Zuko'}, {'name': 'Katie', 'age': 33, 'pet': 'Cisco'}]}


In [47]:
# python->json
asjson = json.dumps(res)
print(asjson)

{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"}, {"name": "Katie", "age": 33, "pet": "Cisco"}]}


In [48]:
# 转dataframe
siblings = DataFrame(res['siblings'], columns=['name','age'])
print(siblings)

    name  age
0  Scott   25
1  Katie   33


## XML和HTML: Web信息收集

In [53]:
parsed = parse(request.urlopen('https://blog.csdn.net/nav/back-end'))
doc = parsed.getroot()
links = doc.findall(".//a")
print(links[15:20])

[<Element a at 0x250d142a040>, <Element a at 0x250d1354db0>, <Element a at 0x250d1354680>, <Element a at 0x250d1354130>, <Element a at 0x250d13544f0>]


In [55]:
# 获取url和链接文本
lnk = links[15]
print(lnk.get("href"))
print(lnk.text_content())

/nav/cloud
云平台


In [56]:
urls = [lnk.get("href") for lnk in doc.findall(".//a")]
print(urls[-10:])

['https://blog.csdn.net/lf21qp', 'https://blog.csdn.net/lf21qp', 'https://blog.csdn.net/QinTao9961220', 'https://blog.csdn.net/QinTao9961220', 'https://blog.csdn.net/wstever', 'https://blog.csdn.net/wstever', 'https://blog.csdn.net/qq_40647372', 'https://blog.csdn.net/qq_40647372', 'https://blog.csdn.net/bishe401', 'https://blog.csdn.net/bishe401']


In [57]:
# 获取表格
tables = doc.findall(".//table")
tb1 = tables[0]
print(tb1)

IndexError: list index out of range

In [ ]:
# 获取表格标题行
rows = tb1.findall(".//tr")


In [ ]:
# 获取表格单元格的文本
def _unpack(row, kind='td'):
    elts = row.findall(".//%s" % kind)
    return [val.text_content() for val in elts]

_unpack(rows[0], kind='th')
_unpack(rows[0], kind='td')

In [ ]:
from pandas.io.parsers import TextParser
def parse_options_data(table):
    rows = table.findall(".//tr")
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    # onverts lists of lists/tuples into DataFrames 
    # with proper type inference and optional 
    # (e.g. string to datetime) conversion. 
    return TextParser(data, names=header).get_chunk()

parse_options_data(tb1)

In [69]:
path = "../resource/mta_perf/Performance_MNR.xml"
parsed = objectify.parse(open(path))
root = parsed.getroot()

data = []

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        el_data[child.tag] = child.pyval
    data.append(el_data)

perf = DataFrame(data)
print(perf[1:3])

   INDICATOR_SEQ PARENT_SEQ           AGENCY_NAME  \
1          28445             Metro-North Railroad   
2          28445             Metro-North Railroad   

                         INDICATOR_NAME  \
1  On-Time Performance (West of Hudson)   
2  On-Time Performance (West of Hudson)   

                                         DESCRIPTION  PERIOD_YEAR  \
1  Percent of commuter trains that arrive at thei...         2008   
2  Percent of commuter trains that arrive at thei...         2008   

   PERIOD_MONTH            CATEGORY FREQUENCY DESIRED_CHANGE INDICATOR_UNIT  \
1             2  Service Indicators         M              U              %   
2             3  Service Indicators         M              U              %   

   DECIMAL_PLACES YTD_TARGET YTD_ACTUAL MONTHLY_TARGET MONTHLY_ACTUAL  
1               1       95.0       96.0           95.0           95.0  
2               1       95.0       96.3           95.0           96.9  


In [75]:
tag = '<a href="http://www.baidu.com">baidu</a>'
print(StringIO(tag))
root = objectify.parse(StringIO(tag)).getroot()
print(root.get("href"))
print(root.text)

http://www.baidu.com
baidu


## 二进制数据格式

In [77]:
frame = pd.read_csv("../resource/ex1.csv")
print(frame)
# 以 pickle形式保存
frame.to_pickle("../resource/frame_pickle")

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [79]:
# 读取
print(pd.read_pickle("../resource/frame_pickle"))

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


## 使用HDF5

HDF：层次型数据格式

先安装 `pip install tables`

适用于:
- 非常大以致于无法放入内存中的数据集
- 一次写多次读的数据集

In [81]:
store = pd.HDFStore('../resource/mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
print(store)

<class 'pandas.io.pytables.HDFStore'>
File path: ../resource/mydata.h5



In [82]:
print(store['obj1'])

   a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo


In [ ]:
import os
store.close()
os.remove('mydata.h5')


## 读取Microsoft Excel文件

pip install openpyxl

pip install xlrd

In [85]:
xls_file = pd.ExcelFile('../resource/ex1.xlsx')
table = xls_file.parse("Sheet1")
print(table)

   Unnamed: 0  a   b   c   d message
0           0  1   2   3   4   hello
1           1  5   6   7   8   world
2           2  9  10  11  12     foo


## 使用HTML和WebAPI

In [ ]:
import requests
url = 'https://search.twitter.com/search.json?q=python%20pandas'
resp = requests.get(url)
print(resp)

In [ ]:
data = json.loads(resp.text)
print(data.keys())

In [ ]:
tweet_fields = ['created_at','from_user','id','text']
tweets = DataFrame(data['results'], columns=tweet_fields)
print(tweets)
print(tweets.iloc[7])

## 使用数据库

In [ ]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [ ]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [ ]:
cursor.description

In [ ]:
DataFrame(rows, columns=zip(*cursor.description)[0])

In [ ]:
import pandas.io.sql as sql
sql.read_sql('select * from test', con)

In [ ]:
import pymongo
con = pymongo.Connection('localhost', port=27017)
tweets = con.db.tweets
url = 'https://search.twitter.com/search.json?q=python%20pandas'
data = json.loads(requests.get(url).text)

for tweet in data['results']:
    tweets.save(tweet)

cursor = tweets.find({'from_user': 'wesmckinn'})
tweet_fields = ['created_at','from_user','id','text']
result = DataFrame(data['results'], columns=tweet_fields)
print(result)
